### 전처리

In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

In [3]:
print(train.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [4]:
train.target[0]

7

In [5]:
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
tfidf = TfidfVectorizer(stop_words='english', max_features=10000)

In [8]:
x_train = tfidf.fit_transform(train.data)

In [9]:
x_train.shape

(11314, 10000)

In [10]:
train.target

array([7, 4, 4, ..., 3, 1, 8])

In [11]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_train = lb.fit_transform(train.target)

In [12]:
y_train[:5]

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [13]:
x_test = tfidf.transform(test.data)
y_test = lb.transform(test.target)

In [14]:
import joblib

joblib.dump(
    {
        'tfidf': tfidf,
        'target_names': train.target_names,
        'x_train': x_train,
        'y_train': y_train,    
        'x_test': x_test,
        'y_test': y_test,
    },
    'newsgroup.pkl')

['newsgroup.pkl']

### 다항분류

In [15]:
import joblib
data = joblib.load('newsgroup.pkl')
locals().update(data)

In [16]:
import tensorflow as tf

In [17]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation='softmax')
])

In [18]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.01),
    metrics=['accuracy'])

In [19]:
model.fit(
    x_train.toarray(),
    y_train,
    epochs=30,
    validation_split=.1,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy')],
)

Epoch 1/30
319/319 [==============================] - 2s 6ms/step - loss: 2.2367 - accuracy: 0.6260 - val_loss: 0.9248 - val_accuracy: 0.8869
Epoch 2/30
319/319 [==============================] - 1s 2ms/step - loss: 0.5909 - accuracy: 0.9625 - val_loss: 0.5956 - val_accuracy: 0.9028
Epoch 3/30
319/319 [==============================] - 1s 2ms/step - loss: 0.2931 - accuracy: 0.9823 - val_loss: 0.4780 - val_accuracy: 0.9064
Epoch 4/30
319/319 [==============================] - 1s 2ms/step - loss: 0.1786 - accuracy: 0.9907 - val_loss: 0.4209 - val_accuracy: 0.9064


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                200020    
Total params: 200,020
Trainable params: 200,020
Non-trainable params: 0
_________________________________________________________________


In [21]:
import pandas as pd

In [22]:
w, _ = model.weights

In [23]:
weights = pd.DataFrame(w.numpy(), columns=target_names, index=tfidf.get_feature_names())

In [24]:
target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [25]:
weights.sort_values('sci.space', ascending=False)[['sci.space']]

,sci.space
space,3.297918
orbit,2.467877
sci,2.151419
moon,1.939135
earth,1.789611
...,...
file,-3.160932
car,-3.211775
hp,-3.427400
windows,-3.772925


In [26]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(20, activation='softmax')
])